In [3]:

import pandas as pd
import numpy
from scipy.stats.mstats import gmean
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn

state_list = [
  "Alabama",
  "Alaska",
  "Arizona",
  "Arkansas",
  "California",
  "Colorado",
  "Connecticut",
  "Delaware",
  "District of Columbia",
  "Florida",
  "Georgia",
  "Hawaii",
  "Idaho",
  "Illinois",
  "Indiana",
  "Iowa",
  "Kansas",
  "Kentucky",
  "Louisiana",
  "Maine",
  "Maryland",
  "Massachusetts",
  "Michigan",
  "Minnesota",
  "Mississippi",
  "Missouri",
  "Montana",
  "Nebraska",
  "Nevada",
  "New Hampshire",
  "New Jersey",
  "New Mexico",
  "New York",
  "North Carolina",
  "North Dakota",
  "Ohio",
  "Oklahoma",
  "Oregon",
  "Pennsylvania",
  "Rhode Island",
  "South Carolina",
  "South Dakota",
  "Tennessee",
  "Texas",
  "Utah",
  "Vermont",
  "Virginia",
  "Washington",
  "West Virginia",
  "Wisconsin",
  "Wyoming",
]

In [11]:
data_1 = pd.read_csv("CAGDP1__ALL_AREAS_2001_2020.csv", encoding="ISO-8859-1")
data_1 = data_1[data_1["Description"] == "Current-dollar GDP (thousands of current dollars)"]
data_1 = data_1[data_1["GeoName"].isin(state_list)]
data_1 = data_1.melt(id_vars=["GeoName"], value_vars=[str(i) for i in range(2001, 2020)], var_name="year",
                     value_name="gdp")
data_1 = data_1.dropna()
data_1 = data_1[data_1["gdp"] != "(NA)"]
data_1["gdp"] = data_1["gdp"].astype("int64")
data_1["year"] = data_1["year"].astype("int")
data_1.head()

data_2 = pd.read_excel("download.xls", skiprows=range(0, 5), nrows=59)
data_2["GeoName"] = data_2["GeoName"].replace("Alaska *", "Alaska")
data_2 = data_2.melt(id_vars=["GeoName"], value_vars=[str(i) for i in range(2008, 2020)], var_name="year",
                     value_name="population")
data_2["year"] = data_2["year"].astype("int")
data_2.head()

data_3 = pd.read_csv("SAIRPDImplicitRegionalPriceDeflatorsbystate.csv")
data_3 = data_3.melt(id_vars=["GeoName"], value_vars=[str(i) for i in range(2008, 2020)], var_name="year",
                     value_name="regional_deflator")
data_3["year"] = data_3["year"].astype("int")
data_3.head()

data = pd.merge(data_1, data_2, on=["GeoName", "year"])
data = pd.merge(data, data_3, on=["GeoName", "year"])
data = data.rename({"GeoName": "state"}, axis=1)
data["gdp_per_capita"] = data["gdp"] / data["population"]
data["gdp_per_capita"] = data["gdp_per_capita"] / (data["regional_deflator"] / 100)
# data["gdp_per_capita"] = numpy.log(data["gdp_per_capita"])
data_Y0 = data[data["year"] == 2008][["state", "gdp_per_capita"]]
data_Y0 = data_Y0.rename(columns={"gdp_per_capita": "Y0"})
data = data.sort_values(["state", "year"])
data["gdp_per_capita_t-1"] = data.groupby("state")["gdp_per_capita"].shift(1)
data["gdp_per_capita_growth"] = data["gdp_per_capita"] / data["gdp_per_capita_t-1"]
# data["gdp_per_capita_diff"] = data["gdp_per_capita"] - data["gdp_per_capita_t-1"]
data = data.dropna()
data.head()

data = data.merge(data_Y0, on=["state"])

data_agg = data.groupby(by="state").agg({"gdp_per_capita_growth": [gmean]}) - 1
data_agg.columns = data_agg.columns.droplevel()
data_agg.head()

data_agg = data_agg.merge(data_Y0, on=["state"])
data_Y0["Y0"] = numpy.log(data_Y0["Y0"])

FileNotFoundError: [Errno 2] No such file or directory: 'CAGDP1__ALL_AREAS_2001_2020.csv'

In [12]:
data_research = pd.read_excel("nsf22307-tab003.xlsx", skiprows=[0, 1, 2])
data_research["State"] = data_research["State"].str.strip()
data_research = data_research.melt(id_vars=["State"], value_vars=[float(i) for i in range(2006, 2020)], var_name="year",
                     value_name="research_spending")
data_research["year"] = data_research["year"].astype(int)
data_research.head()

C:\Users\jiri.pesik.HULD\PycharmProjects\MacroeconomicAnalysis\venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,State,year,research_spending
0,United Statesa,2006,1059104585.0
1,Alabama,2006,7269319.0
2,Alaska,2006,10019060.0
3,Arizona,2006,37151471.0
4,Arkansas,2006,4869648.0


In [14]:
data_labour = pd.read_excel("lpc-by-state-and-region.xlsx", sheet_name="Data (Flat)")
data_labour_hours = data_labour[(data_labour["Measure"] == "Number of hours") & (data_labour["Units"] == "millions")]
data_labour_costs = data_labour[(data_labour["Measure"] == "Labor compensation cost") & (data_labour["Units"] == "million$")]
data_labour_hours.head()


,Area Name,Area Type,Sector,Measure,Units,Year,Value
0,Alabama,State,Private Nonfarm,Number of hours,millions,2007,3149.999
1,Alabama,State,Private Nonfarm,Number of hours,millions,2008,2999.167
2,Alabama,State,Private Nonfarm,Number of hours,millions,2009,2715.492
3,Alabama,State,Private Nonfarm,Number of hours,millions,2010,2727.896
4,Alabama,State,Private Nonfarm,Number of hours,millions,2011,2738
